<a href="https://colab.research.google.com/github/TiagoBarantini/Pro_Eng_dados/blob/main/Prepara%C3%A7%C3%A3o%2C_Orquestra%C3%A7%C3%A3o_e_Fluxos_de_Dados_AULA02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
#instalação JAVA
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

#instalação do pyspark
pip install -q pyspark


In [2]:
#definir uma variavel de ambiente para identificador local do JAVA
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash

#download dos dados utilizados
mkdir titanic
curl https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv -o titanic/titanic.csv

In [4]:
#importar os modulos necessarios

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [5]:
#iniciar uma sessão spark
spark = SparkSession.builder.getOrCreate()

In [ ]:
#leitura de dados
titanic = (
    spark
    .read
    .csv("titanic", header=True, sep=";", inferSchema=True)
  )

titanic.printSchema()

In [ ]:
titanic.show()

In [11]:
#selecionar colunas
titanic.select('Name','Sex','Pclass','Survived').show(n=10, truncate = False)

+---------------------------------------------------+------+------+--------+
|Name                                               |Sex   |Pclass|Survived|
+---------------------------------------------------+------+------+--------+
|Braund, Mr. Owen Harris                            |male  |3     |0       |
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|1     |1       |
|Heikkinen, Miss. Laina                             |female|3     |1       |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|1     |1       |
|Allen, Mr. William Henry                           |male  |3     |0       |
|Moran, Mr. James                                   |male  |3     |0       |
|McCarthy, Mr. Timothy J                            |male  |1     |0       |
|Palsson, Master. Gosta Leonard                     |male  |3     |0       |
|Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  |female|3     |1       |
|Nasser, Mrs. Nicholas (Adele Achem)                |female|2     |1       |

In [14]:
#filtrar linhas
titanic.where("Sex = 'male' AND Survived = 1").show()

+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-------+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|  Ticket|   Fare|  Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-------+--------+
|         18|       1|     2|Williams, Mr. Cha...|male|null|    0|    0|  244373|   13.0|   null|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|male|34.0|    0|    0|  248698|   13.0|    D56|       S|
|         24|       1|     1|Sloper, Mr. Willi...|male|28.0|    0|    0|  113788|   35.5|     A6|       S|
|         37|       1|     3|    Mamee, Mr. Hanna|male|null|    0|    0|    2677| 7.2292|   null|       C|
|         56|       1|     1|   Woolner, Mr. Hugh|male|null|    0|    0|   19947|   35.5|    C52|       S|
|         66|       1|     3|Moubarek, Master....|male|null|    1|    1|    2661|15.2458|   null|       C|
|         75|       1|     3|       B

In [15]:
#calculando metricas MEAN,MIN,MAX,STDDEV
titanic.agg(
    f.mean('Age').alias('med_idade'),
    f.min('Age').alias('idade_minima'),
    f.max('Age').alias('idade_maxima'),
    f.stddev('Age').alias('desvio_padrao_id')
).show()

+-----------------+------------+------------+------------------+
|        med_idade|idade_minima|idade_maxima|  desvio_padrao_id|
+-----------------+------------+------------+------------------+
|29.69911764705882|        0.42|        80.0|14.526497332334035|
+-----------------+------------+------------+------------------+



In [17]:
#metricas agrupadas
(
    titanic
    .groupBy('Sex')
    .agg(
      f.mean('Age').alias('med_idade'),
      f.min('Age').alias('idade_minima'),
      f.max('Age').alias('idade_maxima'),
        f.stddev('Age').alias('desvio_padrao_id'))
).show()

+------+------------------+------------+------------+------------------+
|   Sex|         med_idade|idade_minima|idade_maxima|  desvio_padrao_id|
+------+------------------+------------+------------+------------------+
|female|27.915708812260537|        0.75|        63.0|14.110146457544133|
|  male| 30.72664459161148|        0.42|        80.0|14.678200823816608|
+------+------------------+------------+------------+------------------+



In [21]:
#ordenar resultados

(
    titanic
    .groupBy('Sex','Pclass','Survived')
    .agg(
      f.mean('Age').alias('med_idade'),
      f.min('Age').alias('idade_minima'),
      f.max('Age').alias('idade_maxima'),
      f.stddev('Age').alias('desvio_padrao_id')
        )
    .orderBy('Sex','Pclass','Survived')
    .show()
)

+------+------+--------+------------------+------------+------------+------------------+
|   Sex|Pclass|Survived|         med_idade|idade_minima|idade_maxima|  desvio_padrao_id|
+------+------+--------+------------------+------------+------------+------------------+
|female|     1|       0|25.666666666666668|         2.0|        50.0| 24.00694344004112|
|female|     1|       1|  34.9390243902439|        14.0|        63.0|13.223013572001038|
|female|     2|       0|              36.0|        24.0|        57.0|12.915107432770352|
|female|     2|       1|28.080882352941178|         2.0|        55.0| 12.76469311243358|
|female|     3|       0|23.818181818181817|         2.0|        48.0|12.833464514612233|
|female|     3|       1|19.329787234042552|        0.75|        63.0|12.303245681590393|
|  male|     1|       0| 44.58196721311475|        18.0|        71.0|14.457748881905525|
|  male|     1|       1|36.248000000000005|        0.92|        80.0|14.936743749765334|
|  male|     2|      